In [62]:
import numpy as np 
import pandas as pd 
import nltk
from nltk.corpus import stopwords
import string
from sklearn.naive_bayes import MultinomialNB

In [63]:
df_test = pd.read_excel('testing.xlsx')
df = pd.read_excel('training.xlsx')

In [64]:
#junto los datasets para separar aplicar el modelo a todo y que luego ambos datasets tengan los mismos archivos
frames = [df.text, df_test.text]
completo = pd.concat(frames)
##remove punctation
punc = string.punctuation
def process_text(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    return clean_words

In [65]:
#utilizo el CountVectorizer para limpiar y tokenizar
from sklearn.feature_extraction.text import CountVectorizer
messages_bow = CountVectorizer(analyzer=process_text).fit_transform(completo)

In [66]:
df['text'].head().apply(process_text)
df_test['text'].head().apply(process_text)

0    [Subject, interview, schedule, greg, mikkelson...
1    [Subject, favorite, programs, incredibly, low,...
2    [Subject, mission, impossible, hr, associate, ...
3    [Subject, reply, p, united, trust, bank, limit...
4    [Subject, clewlow, strickland, derivatives, bo...
Name: text, dtype: object

In [67]:
#junto los datasets para separar aplicar el modelo a todo y que luego ambos datasets tengan los mismos archivos
frames = [df.text, df_test.text]
completo = pd.concat(frames)

In [71]:
df

,text,spam
0,"Subject: double coverage amount , same payment...",1
1,Subject: storage meeting ladies and gentlemen...,0
2,"Subject: grades pam , another team : elena ...",0
3,Subject: contact info fyi ! - - - - - - - - ...,0
4,Subject: re : hi : thanks vince : could you ...,0
...,...,...
3995,Subject: california update 3 / 06 / 01 execut...,0
3996,"Subject: aram rick , aram is coming to houst...",0
3997,Subject: re : hello from vince kaminski at enr...,0
3998,"Subject: vince , i ' ll have him e - mail you...",0


In [68]:
#separo los datasets luego de realizar el CountVectorizer
mbow_train = messages_bow[0:4000,]
mbow_test = messages_bow[4000:,]

In [69]:
#del dataset de training, separo en train y test (esta celda es pesada)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(mbow_train, df['spam'], test_size = 0.20, random_state = 0)

In [70]:
#quedaron 37229 columnas, esto es palabras distintas
mbow_train.shape

(4000, 37205)

In [54]:
#entreno modelo
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

In [55]:
#Print the predictions
print(classifier.predict(X_train))

[1 1 0 ... 0 0 0]


In [56]:
#Print the actual values
print(y_train.values)

[1 1 0 ... 0 0 0]


In [57]:
#Evaluate the model on the training data set
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
pred = classifier.predict(X_train)
print(classification_report(y_train ,pred ))
print('Confusion Matrix: \n',confusion_matrix(y_train,pred))
print()
print('Accuracy: ', accuracy_score(y_train,pred))

              precision    recall  f1-score   support

           0       0.92      0.93      0.92      2427
           1       0.76      0.76      0.76       773

    accuracy                           0.89      3200
   macro avg       0.84      0.84      0.84      3200
weighted avg       0.89      0.89      0.89      3200

Confusion Matrix: 
 [[2246  181]
 [ 185  588]]

Accuracy:  0.885625


In [58]:
#Evaluate the model on the test data set
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
pred = classifier.predict(X_test)
print(classification_report(y_test ,pred ))
print('Confusion Matrix: \n', confusion_matrix(y_test,pred))
print()
print('Accuracy: ', accuracy_score(y_test,pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92       615
           1       0.71      0.78      0.74       185

    accuracy                           0.88       800
   macro avg       0.82      0.84      0.83       800
weighted avg       0.88      0.88      0.88       800

Confusion Matrix: 
 [[557  58]
 [ 41 144]]

Accuracy:  0.87625


In [59]:
#Ejecuto modelo sobre el entregable
test = mbow_test
pred = classifier.predict(test)

#genero entregable
submission = pd.DataFrame(list(range(len(df_test))),columns =['Id'])
submission['Category'] = pred
print('Tasa de spam:',(submission.Category==1).mean())
submission.head()

submission.index = submission['Id']
submission.drop(['Id'],axis = 1, inplace=True)

#exporto
submission.to_csv('submission.csv')

Tasa de spam: 0.26851851851851855
